In [7]:
import pandas as pd
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os
import face_recognition
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from PIL import Image

In [8]:
from codes import importPath, resizeImages

# Getting all the images path:

In [9]:
folders = ['0_2', '4_6', '8_12', '15_20', '25_32', '38_43', '48_53', '60_100']

faceFiles = []
    
for files in folders:
    ff = importPath(files)
    faceFiles.append(ff)

## Create a dictionary with folder(label) / images path:

In [10]:
pics = dict()
for ffiles in faceFiles:
    for foto in ffiles:
        label = foto.split("/")[-2]
        pic = foto
        if label in pics:
            pics[label].append(pic)
        else:
            pics[label] = [pic]

## Create a dataframe with the information label / images path:

In [11]:
im = []

for k,v in pics.items():
    for a in v:
        im.append([k,a])
faces_df =  pd.DataFrame(im, columns=["label","image_path"])

display(faces_df.head())

,label,image_path
0,0_2,../OUTPUT/images/0_2/image_669.jpg
1,0_2,../OUTPUT/images/0_2/image_1570.jpg
2,0_2,../OUTPUT/images/0_2/image_346.jpg
3,0_2,../OUTPUT/images/0_2/image_441.jpg
4,0_2,../OUTPUT/images/0_2/image_1055.jpg


In [12]:
faces_df.shape

(17462, 2)

In [13]:
faces_df.label.value_counts()

25_32     5023
0_2       2491
38_43     2340
4_6       2140
8_12      2124
15_20     1642
60_100     872
48_53      830
Name: label, dtype: int64

We might need to delete some of the 25_32 data, so it doesn't interfe in the prediction results.

# Extract face from image:

Import haarcascade:

In [14]:
face_cascade = cv2.CascadeClassifier('../INPUT/haarcascade_frontalface_default.xml')

We will need to treat the images in order to detect the faces:

## Convert images into B&W:

In [17]:
def BlackandWhite(image_path):
    im_bw = cv2.imread(image_path)
    im_bw = cv2.cvtColor(im_bw, cv2.COLOR_BGR2GRAY)
    return im_bw

In [16]:
dictionary={}
count = 0
for path in faces_df['image_path']:
    dictionary[count] = BlackandWhite(path)
    count+=1

TypeError: src is not a numpy array, neither a scalar

# Select all non-facial recognition image:

In [53]:
face_cascade = cv2.CascadeClassifier('../INPUT/haarcascade_frontalface_default.xml')

In [101]:
def loadImageAsGray(imagePath):
    im = cv2.imread(imagePath)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    return im

In [114]:
data = {'Label': '0_2', 
        'path': f1}
result = pd.DataFrame.from_dict(data)

In [115]:
result.head()

,Label,path
0,0_2,../OUTPUT/treated_images/0-2/image_669.jpg
1,0_2,../OUTPUT/treated_images/0-2/image_1570.jpg
2,0_2,../OUTPUT/treated_images/0-2/image_346.jpg
3,0_2,../OUTPUT/treated_images/0-2/image_441.jpg
4,0_2,../OUTPUT/treated_images/0-2/image_1055.jpg


In [117]:
dictionary={}
count = 0
for path in result['path']:
    dictionary[count]=loadImageAsGray(path)
    count+=1

In [123]:
len(f1)

2491

In [165]:
def configImag(array):
    try:
        faces = face_cascade.detectMultiScale(array, 1.25, 6)
        x,y,w,h = faces[0]
        face = array[y:y+h,x:x+w]
        face = cv2.resize(face,(48,48))
        return face
    except:
        return "face not found"

In [166]:
diction={}
for k,v in dictionary.items():
    diction.setdefault('clean_image', []).append(configImag(v))

In [167]:
diction

{'clean_image': ['face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  array([[ 83,  80,  78, ..., 104,  99,  97],
         [ 96,  92,  89, ..., 100,  96,  94],
         [104, 101,  97, ...,  96,  92,  91],
         ...,
         [135, 128, 121, ...,  95,  90,  86],
         [145, 135, 129, ...,  94,  93,  92],
         [144, 134, 132, ...,  94,  95,  96]], dtype=uint8),
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',
  'face not found',


In [168]:
pd.DataFrame(diction).shape

(2491, 1)

In [169]:
pd.DataFrame(diction).isna().sum()

clean_image    0
dtype: int64

In [170]:
result['arrays']= pd.DataFrame(diction)

In [171]:
result['arrays'].isnull().sum()

0

In [173]:
result[result.arrays == 'face not found']

/home/carolina/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:57: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


,Label,path,arrays
0,0_2,../OUTPUT/treated_images/0-2/image_669.jpg,face not found
1,0_2,../OUTPUT/treated_images/0-2/image_1570.jpg,face not found
2,0_2,../OUTPUT/treated_images/0-2/image_346.jpg,face not found
3,0_2,../OUTPUT/treated_images/0-2/image_441.jpg,face not found
4,0_2,../OUTPUT/treated_images/0-2/image_1055.jpg,face not found
...,...,...,...
2486,0_2,../OUTPUT/treated_images/0-2/image_2270.jpg,face not found
2487,0_2,../OUTPUT/treated_images/0-2/image_1103.jpg,face not found
2488,0_2,../OUTPUT/treated_images/0-2/image_461.jpg,face not found
2489,0_2,../OUTPUT/treated_images/0-2/image_472.jpg,face not found


In [174]:
result.drop(result[result.arrays == 'face not found'].index, inplace = True)

/home/carolina/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:57: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [175]:
result

,Label,path,arrays
15,0_2,../OUTPUT/treated_images/0-2/image_1108.jpg,"[[83, 80, 78, 79, 72, 63, 57, 58, 60, 62, 64, ..."
38,0_2,../OUTPUT/treated_images/0-2/image_273.jpg,"[[87, 91, 97, 109, 119, 128, 139, 147, 152, 16..."
49,0_2,../OUTPUT/treated_images/0-2/image_633.jpg,"[[49, 58, 71, 84, 79, 52, 26, 20, 21, 29, 47, ..."
56,0_2,../OUTPUT/treated_images/0-2/image_1964.jpg,"[[13, 12, 17, 51, 82, 108, 107, 108, 111, 120,..."
64,0_2,../OUTPUT/treated_images/0-2/image_1689.jpg,"[[251, 253, 250, 240, 222, 148, 105, 84, 72, 6..."
...,...,...,...
2425,0_2,../OUTPUT/treated_images/0-2/image_2298.jpg,"[[87, 96, 107, 115, 120, 125, 128, 131, 134, 1..."
2427,0_2,../OUTPUT/treated_images/0-2/image_387.jpg,"[[10, 10, 10, 10, 11, 14, 17, 16, 16, 19, 23, ..."
2443,0_2,../OUTPUT/treated_images/0-2/image_2289.jpg,"[[66, 59, 55, 60, 55, 54, 58, 53, 51, 52, 57, ..."
2450,0_2,../OUTPUT/treated_images/0-2/image_1279.jpg,"[[138, 144, 153, 160, 172, 188, 195, 201, 204,..."


# Selected non-detected faces:

In [ ]:
face_cascade = 'haarcascade_frontalface_default.xml'

def transfImag(path):
    print ('transforming image from {}'.format(path))

    input_img=cv2.imread(path)
    input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(input_img, 1.25, 6)
    x,y,w,h = faces[0]
    img_data= input_img[y:y+h,x:x+w]
    img_data=cv2.resize(img_data,(48,48))
    
    img_data = np.stack(img_data)
    img_data = img_data / 255.0
    
    return img_data